In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import the CRUD Python Module for database interaction
from CRUDFiles import CRUDModule



###########################
# Data Manipulation / Model
###########################

# Hardcoded username and password to interact with the animal shelter database
username = "aacuser"
password = "4acUsrp4ss"
shelter = CRUDModule.AnimalShelter(username, password)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Grazioso Salvare Shelter Dashboard')

#Initialize dataframe will all records
df = pd.DataFrame.from_records(shelter.read({}))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    
    html.Center([
        html.A(
            href="https://www.snhu.edu/",
            children=[
            html.Img(
                id='salvare-logo',
                src='assets/Grazioso-Salvare-Logo.png',
                style={'width': '200px', 'height': '200px'})
            ]),
        html.B(html.H1('Austin Shelter Dashboard'))
        
    ]),
 
    
    html.Hr(),
    # A dropdown menu to filter animals based on their suitability as a rescue animal
    html.Div([
        dcc.Dropdown(['All', 'Water Rescue', 'Mountain/Wilderness Rescue', 'Disaster/Individual Tracking'], 
                     'All', id='filter-dropdown'), 
    ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        # Features for interactive data table to make it user-friendly for clients
        # Native sorting allows users to sort the datatable based on each column type
        sort_action='native',
        sort_mode='single',
        # Paging set to have only 20 entries displayed at a time to allow for easier navigation
        page_action='native',
        page_current=0,
        page_size=20,
        row_selectable='single',
        selected_rows=[0],
        # Hides latitude and longitude from table to save space
        hidden_columns=['location_lat', 'location_long'],
        # Hides toggle columns button as lat and long do not need to be shown
        # Lat and long are already visible on geolocation map
        css=[{'selector': '.show-hide', 'rule': 'display: none'}],

    ),
    html.Br(),
    html.Hr(),
    html.Center([ 
        html.Div([
            html.Div(
                    id='map-id',
                    className='col s12 m6',
                    ),
        ], style={'display': 'inline-block'}),
        html.Div([dcc.Graph(id='pie-id',)
                 ], style={'display': 'inline-block', 'width': '875px', 'height': '500px'}),
    ]),
    html.Br(),
    html.Hr(),
    html.Footer('CS-340 Tyler Blanchard', id="footer", style={'font-size': '200%'})
])

#############################################
# Interaction Between Components / Controller
#############################################
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the table based on the selected dropdown option
# Entries listed are based on the preferred breed, sex, and age for each rescue type
# By default, the dataframe/table will gather and display all documents within the animal database
@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-dropdown', 'value')
)
def update_tale(value):
    
    if(value == 'All'): # Default
        # class read method must support return of list object and accept projection json input
        # sending the read method an empty document requests all documents be returned
        df = pd.DataFrame.from_records(shelter.read({}))
    elif(value == 'Water Rescue'): # Gathers/displays animals suited for water rescue
        df = pd.DataFrame.from_records(shelter.read({
            'animal_type' : 'Dog',
            'breed': {'$in': ['Labrador Retriever Mix',
                    'Chesapeake Bay Retriever',
                    'Newfoundland'
                   ]},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }))
    elif(value == 'Mountain/Wilderness Rescue'): # Gathers/displays animals suited for mountain/wild rescue
        df = pd.DataFrame.from_records(shelter.read({
            'animal_type' : 'Dog',
            'breed': {'$in': ['German Shepherd',
                    'Alaskan Malamute',
                    'Old English Sheepdog',
                    'Siberian Husky',
                    'Rottweiler'
                   ]},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }))
    elif(value == 'Disaster/Individual Tracking'): # Gathers/displays animals suited for disaster and individual tracking
        df = pd.DataFrame.from_records(shelter.read({
            'animal_type' : 'Dog',
            'breed': {'$in': ['Doberman Pinscher',
                    'German Shepherd',
                    'Golden Retriever',
                    'Bloodhound',
                    'Rottweiler'
                   ]},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }))

    # MongoDB v5+ is going to return the '_id' column and that is going to have an 
    # invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
    # it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
    # inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
    df.drop(columns=['_id'],inplace=True)
    
    return df.to_dict('records')

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
 # Because we only allow single row selection, the list can 
 # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]

    # Formatted geolocational string for display in map popup
    geo = 'Lat: {lat} Lon: {lon}'.format(lat = dff.iloc[row,13], lon = dff.iloc[row,14])

     # Austin TX is at [30.3,-97.7]
    return [
        dl.Map(style={'width': '875px', 'height': '500px'},
           center=[30.3,-97.7], zoom=9, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for the map
           # Column 4 defines the breed for the animal as location tooltip
           # Column 9 and 11 defines the name and outcome type of the animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                html.H1(dff.iloc[row,9]),
                html.P(dff.iloc[row,11]),
                html.P(geo)
             ])
          ])
       ])
    ]

#This callack will update the pie chart to display the available breeds and amount of said breeds based
#on the selected filter option (default: All)
#To preserve size and quality of the chart, only the largest 10 breeds are listed, with the rest
#being categorized as "Other"
@app.callback(
    Output('pie-id', "figure"),
    Input('datatable-id', "data"))
def update_pie(data):
    pie_colors = ['rgb(128, 0, 0)', 'rgb(153, 0, 0)', 'rgb(153, 51, 0)', 'rgb(204, 51, 0)', 
                  'rgb(153, 0, 51)', 'rgb(204, 0, 0)', 'rgb(255, 0, 0)', 'rgb(255, 51, 0)', 
                  'rgb(204, 0, 102)', 'rgb(255, 80, 80)']
    
    dff = pd.DataFrame.from_dict(data)
    #Computes the breed types and amount for each breed from the active data
    breeds=dff.breed.value_counts()
    #Creates new dataframe to list each breed and the value of each breed
    dfb = pd.DataFrame(breeds)
    dfb = dfb.reset_index()
    dfb.columns = ['Breed', 'Amount']
    dfb.loc[10:, 'Breed'] = 'Other' #Renames all breeds other than the top ten to Other to save space on pie chart
    
    return px.pie(dfb, values='Amount', names='Breed', title='Percentage of Breeds', color_discrete_sequence=pie_colors)

app.run_server()

Dash app running on http://127.0.0.1:13571/
